In [1]:
import numpy as np
import os
import sys
import argparse
import glob
import time
sys.path.append("/home/jungmo/apps/caffe/python") # caffe path

import caffe
from shutil import copyfile

GPUID = 1
BIRD_ROOT = "/home/jungmo/Documents/bird/caffe/"

NPY_PATH = os.path.join(BIRD_ROOT,"model","new_8_200", "mean.npy")
CAFFE_BIN = "/home/jungmo/apps/caffe/build/tools/"
network = 'googLeNet'
IMG_SIZE = 224

def binaryproto_to_npy():
    blob = caffe.proto.caffe_pb2.BlobProto()
    data = open( BINARYPROTO_PATH , 'rb' ).read()
    blob.ParseFromString(data)
    arr = np.array( caffe.io.blobproto_to_array(blob) )
    out = arr[0]
    np.save( NPY_PATH, out )

def evaluate_model(dst):
        input_file = os.path.join(TEST)
        output_file = os.path.join(BIRD_ROOT, "db/output.npy")
        model_def = os.path.join(BIRD_ROOT, "model_prototxt/"+network+"/deploy.prototxt")
        pretrained_model = os.path.join(BIRD_ROOT, "model/new_8_200/googlenet_iter_60000.caffemodel")
        gpu = True
        images_dim='224,224'
        mean_file = NPY_PATH
        ext='bmp'
        channel_swap='2,1,0'
        raw_scale = 255.0
        center_only=True
        input_scale = None
        image_dims = [int(s) for s in images_dim.split(',')]

        mean, channel_swap = None, None
        if mean_file:
            mean = np.load(mean_file)
        if channel_swap:
            channel_swap=[int(s) for s in args.channel_swap.split(',')]
        caffe.set_mode_gpu()
        #caffe.set_device(GPUID)
        # Make classifier.
        classifier = caffe.Classifier(model_def, pretrained_model,
                image_dims=image_dims, mean=mean,
                input_scale=input_scale, raw_scale=raw_scale,
                channel_swap=channel_swap)

        # Load numpy array (.npy), directory glob (*.jpg), or image file.
        imgs_list = tuple(open(input_file,'r'))
        print("Loading file: %s" % input_file)
        inputs =[caffe.io.load_image(im_f)
                     for im_f in glob.glob(input_file + '/*.' + ext)]

        print("Classifying %d inputs." % len(imgs_list))

        BATCH_SIZE = 100

        # Classify.
        confusion_matrix = np.zeros((4,4),dtype=np.int32)
        accuracy = 0.0
        start = time.time()
        for i in range(0, len(imgs_list), BATCH_SIZE):
            if i > 0 and i % (BATCH_SIZE * 25) == 0:
                print("Finished %d imgs." % (i))
                cur_accuracy = accuracy / i * 100
                print("Accuracy: %.2f " % cur_accuracy)
                print("Confusion Matrix:")
                print confusion_matrix
            imgs = [imgs_list[i + j].split(' ')[0] for j in range(min(BATCH_SIZE,len(imgs_list)-i))]
            gts = [int(imgs_list[i + j].split(' ')[1]) for j in range(min(BATCH_SIZE,len(imgs_list)-i))]
            inputs = [caffe.io.load_image(path) for path in imgs]
            predictions = classifier.predict(inputs, not center_only)
            for j in range(len(imgs)):
                probs = predictions[j]
                pred = np.argmax(probs)
                confusion_matrix[gts[j]][pred] += 1
                if pred == gts[j]:
                    accuracy += 1.0

        accuracy = accuracy / len(imgs_list) * 100

        print("Done in %.2f s." % (time.time() - start))
        # Save
        print("Accuracy: %.2f " % accuracy)
        print("Confusion Matrix:")
        print confusion_matrix
        np.save(dst, confusion_matrix)

In [2]:

'''    
print "Make DB : Training set"
os.system(os.path.join(CAFFE_BIN,"convert_imageset.bin")+" -backend=\"lmdb\" -shuffle=true -resize_height "+str(IMG_SIZE)+" -resize_width "+str(IMG_SIZE)+ " / " + TRAIN + " " + BIRD_ROOT + "db/train_imageData_lmDB")

print "Make DB : Test set"
os.system(os.path.join(CAFFE_BIN,"convert_imageset.bin")+" -backend=\"lmdb\" -shuffle=true -resize_height "+str(IMG_SIZE)+" -resize_width "+str(IMG_SIZE) + " / " + TEST + " " + BIRD_ROOT + "db/val_imageData_lmDB")

print "Make image mean file"
os.system(os.path.join(CAFFE_BIN,"compute_image_mean.bin")+
               " -backend=lmdb "+ BIRD_ROOT + "db/train_imageData_lmDB " + BIRD_ROOT+ "db/mean.binaryproto")
print "Image mean file .binaryproto to .npy"
binaryproto_to_npy()   

print "Training model"
os.system(os.path.join(CAFFE_BIN,"caffe.bin")+" train "
               "-solver="+BIRD_ROOT+"model/"+network+"/solver.prototxt "+" -weights="+BIRD_ROOT+"model/"+network+"/bvlc_googlenet.caffemodel "+ "-gpu all")
'''
print "Evaluate model"
TEST = "/home/jungmo/Documents/bird/caffe/bird_no_adole/test/test_8_200.txt"
evaluate_model("test_8_200.npy")


Evaluate model
Loading file: /home/jungmo/Documents/bird/caffe/bird_no_adole/test/test_8_200.txt
Classifying 21586 inputs.
Finished 2500 imgs.
Accuracy: 96.40 
Confusion Matrix:
[[2410   23   31   36]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 5000 imgs.
Accuracy: 96.48 
Confusion Matrix:
[[4824   48   54   74]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 7500 imgs.
Accuracy: 96.27 
Confusion Matrix:
[[6501   57   80  100]
 [  14  719    5   24]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 10000 imgs.
Accuracy: 96.07 
Confusion Matrix:
[[8628   63   97  181]
 [  14  979    8   30]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 12500 imgs.
Accuracy: 93.84 
Confusion Matrix:
[[9566   65  107  229]
 [  14  979    8   30]
 [   9   37 1185  271]
 [   0    0    0    0]]
Finished 15000 imgs.
Accuracy: 91.80 
Confusion Matrix:
[[9566   65  107  229]
 [  14  979    8   30]
 [  17   95 3225  665]
 [   0    

In [3]:
TEST = "/home/jungmo/Documents/bird/caffe/bird_no_adole/test/test_8_150.txt"
evaluate_model("test_8_150.npy")

Loading file: /home/jungmo/Documents/bird/caffe/bird_no_adole/test/test_8_150.txt
Classifying 21586 inputs.
Finished 2500 imgs.
Accuracy: 97.52 
Confusion Matrix:
[[2438    5   35   22]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 5000 imgs.
Accuracy: 97.42 
Confusion Matrix:
[[4871   15   72   42]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 7500 imgs.
Accuracy: 96.85 
Confusion Matrix:
[[6560   19  101   58]
 [  34  704   14   10]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 10000 imgs.
Accuracy: 96.10 
Confusion Matrix:
[[8664   22  152  131]
 [  52  946   18   15]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 12500 imgs.
Accuracy: 95.27 
Confusion Matrix:
[[9611   23  170  163]
 [  52  946   18   15]
 [   8    0 1352  142]
 [   0    0    0    0]]
Finished 15000 imgs.
Accuracy: 94.31 
Confusion Matrix:
[[9611   23  170  163]
 [  52  946   18   15]
 [  31    5 3589  377]
 [   0    0    0    0]]
F

In [ ]:
TEST = "/home/jungmo/Documents/bird/caffe/bird_no_adole/test/test_8_140.txt"
evaluate_model("test_8_140.npy")

Loading file: /home/jungmo/Documents/bird/caffe/bird_no_adole/test/test_8_140.txt
Classifying 21586 inputs.
Finished 2500 imgs.
Accuracy: 96.80 
Confusion Matrix:
[[2420    7   48   25]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 5000 imgs.
Accuracy: 97.16 
Confusion Matrix:
[[4858   17   84   41]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 7500 imgs.
Accuracy: 96.60 
Confusion Matrix:
[[6548   20  112   58]
 [  38  697   18    9]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 10000 imgs.
Accuracy: 95.82 
Confusion Matrix:
[[8642   22  174  131]
 [  58  940   19   14]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 12500 imgs.
Accuracy: 94.93 
Confusion Matrix:
[[9572   23  202  170]
 [  58  940   19   14]
 [   7    1 1354  140]
 [   0    0    0    0]]
Finished 15000 imgs.
Accuracy: 93.66 
Confusion Matrix:
[[9572   23  202  170]
 [  58  940   19   14]
 [  27    8 3537  430]
 [   0    0    0    0]]
F

In [12]:
TEST = "/home/jungmo/Documents/bird/caffe/bird_no_adole/test/test_8_130.txt"
evaluate_model("test_8_130.npy")

Loading file: /home/jungmo/Documents/bird/caffe/bird_no_adole/test/test_8_130.txt
Classifying 21586 inputs.
Finished 2500 imgs.
Accuracy: 96.92 
Confusion Matrix:
[[2423    6   48   23]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 5000 imgs.
Accuracy: 96.96 
Confusion Matrix:
[[4848   16   88   48]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 7500 imgs.
Accuracy: 96.13 
Confusion Matrix:
[[6526   20  123   69]
 [  43  684   18   17]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 10000 imgs.
Accuracy: 95.32 
Confusion Matrix:
[[8601   21  196  151]
 [  56  931   21   23]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 12500 imgs.
Accuracy: 94.66 
Confusion Matrix:
[[9529   23  231  184]
 [  56  931   21   23]
 [  12    8 1372  110]
 [   0    0    0    0]]
Finished 15000 imgs.
Accuracy: 94.15 
Confusion Matrix:
[[9529   23  231  184]
 [  56  931   21   23]
 [  25   12 3663  302]
 [   0    0    0    0]]
F

In [9]:
TEST = "/home/jungmo/Documents/bird/caffe/bird_no_adole/test/test_8_120.txt"
evaluate_model("test_8_120.npy")

Loading file: /home/jungmo/Documents/bird/caffe/bird_no_adole/test/test_8_120.txt
Classifying 21586 inputs.
Finished 2500 imgs.
Accuracy: 95.64 
Confusion Matrix:
[[2391   10   70   29]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 5000 imgs.
Accuracy: 96.06 
Confusion Matrix:
[[4803   15  126   56]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 7500 imgs.
Accuracy: 95.49 
Confusion Matrix:
[[6480   15  169   74]
 [  39  682   15   26]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 10000 imgs.
Accuracy: 93.01 
Confusion Matrix:
[[8381   18  327  243]
 [  53  920   21   37]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 12500 imgs.
Accuracy: 92.74 
Confusion Matrix:
[[9245   18  383  321]
 [  53  920   21   37]
 [  12    2 1428   60]
 [   0    0    0    0]]
Finished 15000 imgs.
Accuracy: 93.03 
Confusion Matrix:
[[9245   18  383  321]
 [  53  920   21   37]
 [  40    3 3789  170]
 [   0    0    0    0]]
F

In [10]:
TEST = "/home/jungmo/Documents/bird/caffe/bird_no_adole/test/test_8_110.txt"
evaluate_model("test_8_110.npy")

Loading file: /home/jungmo/Documents/bird/caffe/bird_no_adole/test/test_8_110.txt
Classifying 21586 inputs.
Finished 2500 imgs.
Accuracy: 96.84 
Confusion Matrix:
[[2421    6   56   17]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 5000 imgs.
Accuracy: 96.72 
Confusion Matrix:
[[4836   10  108   46]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 7500 imgs.
Accuracy: 95.76 
Confusion Matrix:
[[6522   13  141   62]
 [  56  660   20   26]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 10000 imgs.
Accuracy: 93.67 
Confusion Matrix:
[[8472   16  269  212]
 [  72  895   29   35]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 12500 imgs.
Accuracy: 93.47 
Confusion Matrix:
[[9336   16  333  282]
 [  72  895   29   35]
 [  13    0 1453   36]
 [   0    0    0    0]]
Finished 15000 imgs.
Accuracy: 93.87 
Confusion Matrix:
[[9336   16  333  282]
 [  72  895   29   35]
 [  36    1 3850  115]
 [   0    0    0    0]]
F

In [11]:
TEST = "/home/jungmo/Documents/bird/caffe/bird_no_adole/test/test_8_100.txt"
evaluate_model("test_8_100.npy")

Loading file: /home/jungmo/Documents/bird/caffe/bird_no_adole/test/test_8_100.txt
Classifying 21586 inputs.
Finished 2500 imgs.
Accuracy: 97.20 
Confusion Matrix:
[[2430    3   46   21]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 5000 imgs.
Accuracy: 97.44 
Confusion Matrix:
[[4872    4   81   43]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 7500 imgs.
Accuracy: 96.32 
Confusion Matrix:
[[6576    7  106   49]
 [  67  648   24   23]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 10000 imgs.
Accuracy: 94.78 
Confusion Matrix:
[[8604    9  216  140]
 [  86  874   35   36]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 12500 imgs.
Accuracy: 94.56 
Confusion Matrix:
[[9499    9  258  201]
 [  86  874   35   36]
 [  10    2 1447   43]
 [   0    0    0    0]]
Finished 15000 imgs.
Accuracy: 94.73 
Confusion Matrix:
[[9499    9  258  201]
 [  86  874   35   36]
 [  37    5 3836  124]
 [   0    0    0    0]]
F